In [30]:
import requests
import pandas as pd
from datetime import datetime
import sqlite3
import os
from tabulate import tabulate

In [ ]:
class IngestorAPI():

    def __init__(self, url, to_stop):
        self.url = url
        self.to_stop = datetime.strptime(to_stop, "%Y-%m-%d")

    def get_resp(self, **kwargs):
        resp = resp = requests.get(self.url, params=kwargs)
        return resp
    
    def save_data(self, data):
        path = 'C:/Users/LEONARDO/OneDrive/Documentos/Python/Estudos/Dota/raw/pro_matches.csv'
        df = pd.DataFrame(data)
        if  os.path.exists(path):
            df.to_csv(path, mode='a', header=False, sep=';', index=False)
        else:
            df.to_csv(path, mode='d', header=True, sep=';', index=False)

    def get_and_save(self, **kwargs):
        resp = self.get_resp(**kwargs)
        data = resp.json()
        self.save_data(data)
        return data
    
    def auto_process(self):
        data = self.get_and_save()
        df = pd.DataFrame(data)
        less_than_match_id = df['match_id'].min()
        min_start_time = datetime.fromtimestamp(df['start_time'].min())
        while min_start_time > self.to_stop and len(data)==100:
            df = pd.DataFrame(self.get_and_save(less_than_match_id=less_than_match_id))
            less_than_match_id = df['match_id'].min()
            min_start_time = datetime.fromtimestamp(df['start_time'].min())

In [ ]:
url = 'https://api.opendota.com/api/proMatches'

ing = IngestorAPI(url, '2024-11-11')
ing.auto_process()

In [33]:
df = pd.read_csv('pro_matches.csv', sep=';')
df

,match_id,duration,start_time,radiant_team_id,radiant_name,dire_team_id,dire_name,leagueid,league_name,series_id,series_type,radiant_score,dire_score,radiant_win,version
0,8031235309,1356,1731345610,8629324.0,Swift Knights,9425656.0,LSG,17233,Destiny League,924890,1,23,35,False,NaN
1,8031199890,1576,1731344313,NaN,NaN,9600141.0,NaN,17299,ESL One Bangkok 2024 Qualifiers powered by Intel,0,0,24,9,True,NaN
2,8031187982,737,1731343884,NaN,NaN,7542890.0,NaN,17299,ESL One Bangkok 2024 Qualifiers powered by Intel,0,0,18,4,True,21.0
3,8031179918,1318,1731343670,9425656.0,LSG,8629324.0,Swift Knights,17233,Destiny League,924890,1,27,10,True,21.0
4,8031176540,928,1731343481,NaN,NaN,9600476.0,NaN,17299,ESL One Bangkok 2024 Qualifiers powered by Intel,0,0,25,2,True,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6295,8027712452,1987,1731179340,8255888.0,BetBoom Team,7119388.0,Team Spirit,17272,DreamLeague Season 24 powered by Intel,924214,1,20,9,True,21.0
6296,8027585898,1927,1731174845,8761147.0,tam_sme,9571312.0,Casually masculine,17170,Dota Masters Bulgaria,924186,1,25,18,True,21.0
6297,8027574681,2476,1731174458,9530716.0,Air Defence,9120549.0,Rakuzan,17366,Ultras Dota Pro League 2024-25,924206,1,35,50,False,21.0
6298,8027500320,4390,1731173100,8255888.0,BetBoom Team,7119388.0,Team Spirit,17272,DreamLeague Season 24 powered by Intel,924214,1,33,47,False,21.0


In [34]:
# Conecte ao SQLite em memória
conn = sqlite3.connect(":memory:")

# Carregue o DataFrame como uma tabela SQL temporária
df.to_sql("dota", conn, index=False, if_exists="replace")

# Faça uma consulta SQL
query = "SELECT * FROM dota WHERE match_id = 8031235309	"
resultado = pd.read_sql_query(query, conn)

# Exiba o resultado
print(tabulate(resultado, headers='keys', tablefmt='fancy_grid'))

# Feche a conexão
conn.close()

╒════╤════════════╤════════════╤══════════════╤═══════════════════╤════════════════╤════════════════╤═════════════╤════════════╤════════════════╤═════════════╤═══════════════╤═════════════════╤══════════════╤═══════════════╤═══════════╕
│    │   match_id │   duration │   start_time │   radiant_team_id │ radiant_name   │   dire_team_id │ dire_name   │   leagueid │ league_name    │   series_id │   series_type │   radiant_score │   dire_score │   radiant_win │   version │
╞════╪════════════╪════════════╪══════════════╪═══════════════════╪════════════════╪════════════════╪═════════════╪════════════╪════════════════╪═════════════╪═══════════════╪═════════════════╪══════════════╪═══════════════╪═══════════╡
│  0 │ 8031235309 │       1356 │   1731345610 │       8.62932e+06 │ Swift Knights  │    9.42566e+06 │ LSG         │      17233 │ Destiny League │      924890 │             1 │              23 │           35 │             0 │       nan │
├────┼────────────┼────────────┼──────────────┼─────